### Importation des packages

In [25]:
import requests
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import json
import requests
from bs4 import BeautifulSoup
from unidecode import unidecode
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time


### Récupération des liens sur la page principale

In [2]:
url = "https://freeglisse.com/fr/12-ski-occasion"
reponse = requests.get(url)
html_content = reponse.text
soup = BeautifulSoup(html_content, "html.parser")
soup

<!DOCTYPE html>

<html lang="fr-FR">
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<title>Ski occasion - Ski d'occasion - Skis occasions</title>
<meta content="Ski occasion" name="description"/>
<meta content="ski occasion" name="keywords"/>
<link href="https://freeglisse.com/fr/12-ski-occasion" rel="canonical"/>
<link href="https://freeglisse.com/fr/12-ski-occasion" hreflang="fr-fr" rel="alternate"/>
<link href="https://freeglisse.com/de/12-gebrauchte-ski" hreflang="de-de" rel="alternate"/>
<link href="https://freeglisse.com/en/12-used-ski" hreflang="en-en" rel="alternate"/>
<link href="https://freeglisse.com/es/12-esqui-usado" hreflang="es-es" rel="alternate"/>
<link href="https://freeglisse.com/it/12-sci-usato" hreflang="it-it" rel="alternate"/>
<script type="application/ld+json">
  {
    "@context": "https://schema.org",
    "@type": "Organization",
    "name" : "Freeglisse.com",
    "url" : "https://freeglisse.com/fr/",
    "logo": {
      "

In [14]:
# afficher une page
def get_products():
	links_list = []
	url = 'https://freeglisse.com/fr/12-ski-occasion?page=1'
	response = requests.get(url)
	soup = BeautifulSoup(response.content, "html.parser")
	h2_tags = soup.find_all('h2', class_ = 'h3 product-title')
	for ski in h2_tags:
		product_links = [ski.a.get("href") for ski in h2_tags]
	links_list.extend(product_links)
	return links_list
get_products()


['https://freeglisse.com/fr/mini-ski-occasion-adulte/18977-558749-mini-ski-occasion-snowblade-toutes-marques-tous-modeles-fixations.html#/9-etat_du_materiel-qualite_b/832-taille_skis-90_cm',
 'https://freeglisse.com/fr/ski-occasion-adulte-freeride-et-freestyle/18916-431280-ski-occasion-rossignol-sender-104-ti-2024-fixations.html#/3-etat_du_materiel-qualite_a/906-taille_skis-164_cm',
 'https://freeglisse.com/fr/ski-occasion-freeride/18915-431270-ski-occasion-rossignol-sender-94-ti-fixations.html#/3-etat_du_materiel-qualite_a/898-taille_skis-156_cm',
 'https://freeglisse.com/fr/ski-occasion-junior-loisir/19989-553260-ski-occasion-junior-rossignol-frozen-fixations.html#/3-etat_du_materiel-qualite_a/846-taille_skis-104_cm',
 'https://freeglisse.com/fr/ski-occasion-junior-loisir/16949-543945-ski-occasion-junior-rossignol-experience-pro-fixations.html#/3-etat_du_materiel-qualite_a/812-taille_skis-70_cm',
 'https://freeglisse.com/fr/ski-de-fond-occasion-alternatif-sns-profil/18974-434106-ski-

In [24]:
def get_links():
    base_url = "https://freeglisse.com/fr/12-ski-occasion/s-1/?page={}"
    current_page = 1

    links_list = []

    while True:
        url = base_url.format(current_page)

        response = requests.get(url)
        if response.status_code != 200:
            print("Erreur lors de la requête HTTP")
            break

        soup = BeautifulSoup(response.text, 'html.parser')

        h2_tags = soup.find_all("h2", {"class": "h3 product-title"})
        product_links = [h2_tag.a.get("href") for h2_tag in h2_tags]

        links_list.extend(product_links)

        next_page_link = soup.find("a", {"rel": "next", "class": "next js-search-link"})
        if not next_page_link:
            break

        current_page += 1

    return links_list

liste_lien = get_links()
print(liste_lien)
(len(liste_lien))

KeyboardInterrupt: 

### Récupération des informations sur UNE page produit

In [1]:
#Test de récupération sur une page produit
url_t = "https://freeglisse.com/fr/ski-occasion-femme-loisir/18890-430528-ski-occasion-rossignol-nova-6-fixations.html#/3-etat_du_materiel-qualite_a/891-taille_skis-149_cm"
reponse_t = requests.get(url_t)
html_content_t = reponse_t.text
soup_t = BeautifulSoup(html_content_t, "html.parser")

# pour obtenir la RÉFÉRENCE produit
ref_produit = (soup_t.find("div", {"class": "product-reference rb-tag-cate"}).text.strip()).replace("announcement Référence: ", "").strip()
print(ref_produit)
# pour obtenir le NOM du produit
nom_produit = soup_t.find_all("h1", {"class" : "h1 product-detail-name"})[0].get_text().strip()
print(nom_produit)
# pour obtenir la MARQUE du produit
marque_produit = soup_t.find("img", {"class" : "img img-thumbnail manufacturer-logo"}).get("alt", "")
print(marque_produit)
# pour obtenir le PRIX du produit
prix_produit = unidecode(soup_t.find_all("span", {"class" : "current-price-value"})[0].get_text().strip())
print(prix_produit)
# pour obtenir le TYPE DE PRATIQUE du produit
type_pratique = [soup_t.find_all("dd", {"class" : "value"})[0].get_text().strip()]
print(type_pratique)
# pour obtenir le TYPE D'UTILISATEUR du produit
utilisateur = [soup_t.find_all("dd", {"class" : "value"})[1].get_text().strip()]
print(utilisateur)
# pour obtenir le NIVEAU du produit
niveau = [soup_t.find_all("dd", {"class" : "value"})[2].get_text().strip()]
print(niveau)
# pour obtenir la COULEUR du produit
colori = [soup_t.find_all("dd", {"class" : "value"})[3].get_text().strip()]
print(colori)
# pour obtenir le NOMBRE D'AVIS du produit
nb_avis = soup_t.find("p", {"class" : "netreviews_subtitle"}).find("span", {"id": "reviewCount"}).text.strip()
print(nb_avis)
# pour obtenir la NOTE du produit
note_produit = soup_t.find("p", {"class" : "netreviews_note_generale"}).text.strip().split('/')[0]
print(note_produit)

NameError: name 'requests' is not defined

#### Pour obtenir le nombre de produit disponible :

In [ ]:

# Initialiser le driver (ajustez pour votre propre navigateur)
driver = webdriver.Chrome()

# Ouvrir la page du produit
driver.get("https://freeglisse.com/fr/ski-de-fond-occasion-alternatif-norme-sns/15483-167626-ski-de-fond-occasion-toutes-marques-fixation-vieux-sns.html#/882-taille_skis-140_cm/1768-etat_du_materiel-qualite_c")

# Attendre que la page soit complètement chargée
wait = WebDriverWait(driver, 10)

# Localiser le champ de quantité
quantity_input = wait.until(EC.visibility_of_element_located((By.ID, "quantity_wanted")))

# Variable pour stocker le maximum de produits en stock
max_stock = 1  # Commencer avec une quantité de 1

while True:
    # Défiler pour que l'élément de quantité soit visible
    driver.execute_script("arguments[0].scrollIntoView(true);", quantity_input)
    time.sleep(0.5)  # Petite pause pour s'assurer que l'élément est visible

    # Utiliser JavaScript pour définir la valeur du champ de quantité
    driver.execute_script(f"arguments[0].value = {max_stock};", quantity_input)
    
    # Simuler un événement 'change' sur le champ de quantité
    driver.execute_script("var event = new Event('change', { bubbles: true }); arguments[0].dispatchEvent(event);", quantity_input)
    
    # Attendre un peu pour que la page traite l'événement
    time.sleep(2)  # Ajustez cette durée si nécessaire

    # Rechercher le message de disponibilité à chaque itération
    try:
        # Localiser à nouveau le message de disponibilité
        availability_message = wait.until(
            EC.visibility_of_element_located((By.XPATH, "//span[@id='availability_message']"))
        )
        availability_text = availability_message.text
        print(f"Message de disponibilité : {availability_text}")

        # Vérifier si le stock est suffisant
        if "pas assez" in availability_text.lower():
            print(f"Stock maximum détecté avant l'erreur : {max_stock - 1}")
            break
    except Exception as e:
        print(f"Erreur lors de la vérification du message : {str(e)}")
        break
    
    # Si le message est valide, augmenter la quantité
    max_stock += 1

# Fermer le navigateur
driver.quit()

In [16]:
def get_all_pages(base_url):
    urls = []
    current_page = 1
    
    while True:
        full_url = f"{base_url}{current_page}"
        response = requests.get(full_url)
        
        if response.status_code != 200:
            break
        
        soup = BeautifulSoup(response.text, 'html.parser')
        urls.append(full_url)
        
        next_page_link = soup.find("a", {"rel": "next", "class": "next js-search-link"})
        if not next_page_link:
            break
        
        current_page += 1
    
    return urls

qualiteA = get_all_pages('https://freeglisse.com/fr/12-ski-occasion/s-1/etat_du_materiel-qualite_a?page=')
qualiteB = get_all_pages('https://freeglisse.com/fr/12-ski-occasion/s-1/etat_du_materiel-qualite_b?page=')
qualiteC = get_all_pages('https://freeglisse.com/fr/12-ski-occasion/s-1/etat_du_materiel-qualite_c?page=')

In [20]:
qualiteA

['https://freeglisse.com/fr/12-ski-occasion/s-1/etat_du_materiel-qualite_a?page=1',
 'https://freeglisse.com/fr/12-ski-occasion/s-1/etat_du_materiel-qualite_a?page=2',
 'https://freeglisse.com/fr/12-ski-occasion/s-1/etat_du_materiel-qualite_a?page=3',
 'https://freeglisse.com/fr/12-ski-occasion/s-1/etat_du_materiel-qualite_a?page=4',
 'https://freeglisse.com/fr/12-ski-occasion/s-1/etat_du_materiel-qualite_a?page=5',
 'https://freeglisse.com/fr/12-ski-occasion/s-1/etat_du_materiel-qualite_a?page=6',
 'https://freeglisse.com/fr/12-ski-occasion/s-1/etat_du_materiel-qualite_a?page=7',
 'https://freeglisse.com/fr/12-ski-occasion/s-1/etat_du_materiel-qualite_a?page=8',
 'https://freeglisse.com/fr/12-ski-occasion/s-1/etat_du_materiel-qualite_a?page=9',
 'https://freeglisse.com/fr/12-ski-occasion/s-1/etat_du_materiel-qualite_a?page=10',
 'https://freeglisse.com/fr/12-ski-occasion/s-1/etat_du_materiel-qualite_a?page=11',
 'https://freeglisse.com/fr/12-ski-occasion/s-1/etat_du_materiel-qualite_a

### Obtention de tous les liens disponibles sur le site


Afin de récupérer tous les liens des skis, nous allons faire une boucle par qualité (A,B et C). Il fautdrai également obtenir les liens en fonction des tailles (ce qui n'est pas encore fait ici)

#### Table qualité A

In [26]:
url_a = "https://freeglisse.com/fr/12-ski-occasion/s-1/etat_du_materiel-qualite_a"
reponse_a = requests.get(url_a)
html_content_a = reponse_a.text
soup_a = BeautifulSoup(html_content_a, "html.parser")

def table_qualite_a():
	base_url = "https://freeglisse.com/fr/12-ski-occasion/s-1/etat_du_materiel-qualite_a?page={}&order=product.price.desc"
	current_page = 1

	df_global_a = pd.DataFrame(columns=['nom', 'prix', 'lien'])

	while True:
		url = base_url.format(current_page)

		response = requests.get(url)
		if response.status_code != 200:
			print("Erreur lors de la requête HTTP")
			break

		soup = BeautifulSoup(response.text, 'html.parser')

		balises_h2 = soup.find_all("h2", {"class": "h3 product-title"})
		noms_produits = [balise_h2.a.text.strip() for balise_h2 in balises_h2]

		prices = [unidecode(span.get_text().strip()) for span in soup.find_all("span", {"class": "price"})]

		product_links = [h2_tag.a.get("href") for h2_tag in balises_h2]

		data = {'nom': noms_produits, 'prix': prices, 'lien': product_links}
		df_current = pd.DataFrame(data)

		df_global_a = pd.concat([df_global_a, df_current], ignore_index=True)

		next_page_link = soup.find("a", {"rel": "next", "class": "next js-search-link"})
		if not next_page_link:
			break

		current_page += 1
	return df_global_a

df_a = table_qualite_a()

df_a['Qualité'] = 'Qualité A'
df_a

,nom,prix,lien,Qualité
0,Ski occasion Blizzard Anomaly 102 + fixations,"649,00 EUR",https://freeglisse.com/fr/ski-test-all-mountai...,Qualité A
1,Ski occasion Blizzard Anomaly 94 + fixations,"629,00 EUR",https://freeglisse.com/fr/ski-test-all-mountai...,Qualité A
2,Ski occasion Blizzard Anomaly 84 + fixations,"549,00 EUR",https://freeglisse.com/fr/ski-test-all-mountai...,Qualité A
3,Ski occasion Blizzard Thunderbird R15 + fixations,"519,00 EUR",https://freeglisse.com/fr/ski-test-piste/20267...,Qualité A
4,Ski occasion Blizzard Thunderbird R15 WB + fix...,"499,00 EUR",https://freeglisse.com/fr/ski-test-piste/20269...,Qualité A
...,...,...,...,...
291,Ski occasion junior Elan XRJ + fixations,"39,00 EUR",https://freeglisse.com/fr/ski-occasion-junior-...,Qualité A
292,Ski occasion junior Dynastar Team Speed 65 + f...,"39,00 EUR",https://freeglisse.com/fr/ski-occasion-junior-...,Qualité A
293,Ski occasion junior Rossignol Princess + fixat...,"39,00 EUR",https://freeglisse.com/fr/ski-occasion-junior-...,Qualité A
294,Ski occasion junior Dynastar Starlett + Fixations,"39,00 EUR",https://freeglisse.com/fr/ski-occasion-junior-...,Qualité A


#### Table qualité B

In [27]:
url_b = "https://freeglisse.com/fr/12-ski-occasion/s-1/etat_du_materiel-qualite_b"
reponse_b = requests.get(url_b)
html_content_b = reponse_b.text
soup_b = BeautifulSoup(html_content_b, "html.parser")

def table_qualite_b():
	base_url = "https://freeglisse.com/fr/12-ski-occasion/s-1/etat_du_materiel-qualite_b?page={}&order=product.price.desc"
	current_page = 1

	df_global_b = pd.DataFrame(columns=['nom', 'prix', 'lien'])

	while True:
		url = base_url.format(current_page)

		response = requests.get(url)
		if response.status_code != 200:
			print("Erreur lors de la requête HTTP")
			break

		soup = BeautifulSoup(response.text, 'html.parser')

		balises_h2 = soup.find_all("h2", {"class": "h3 product-title"})
		noms_produits = [balise_h2.a.text.strip() for balise_h2 in balises_h2]

		prices = [unidecode(span.get_text().strip()) for span in soup.find_all("span", {"class": "price"})]

		product_links = [h2_tag.a.get("href") for h2_tag in balises_h2]

		data = {'nom': noms_produits, 'prix': prices, 'lien': product_links}
		df_current = pd.DataFrame(data)

		df_global_b = pd.concat([df_global_b, df_current], ignore_index=True)

		next_page_link = soup.find("a", {"rel": "next", "class": "next js-search-link"})
		if not next_page_link:
			break

		current_page += 1
	return df_global_b

df_b = table_qualite_b()

df_b['Qualité'] = 'Qualité B'
df_b

,nom,prix,lien,Qualité
0,Ski occasion Fischer Rc4 The Curv DTX + fixations,"399,00 EUR",https://freeglisse.com/fr/ski-occasion-adulte-...,Qualité B
1,Ski occasion Kastle FX 96 + fixations,"349,00 EUR",https://freeglisse.com/fr/ski-occasion-freerid...,Qualité B
2,Ski occasion Elan Insomnia Black Edition + fix...,"339,00 EUR",https://freeglisse.com/fr/ski-occasion-femme-p...,Qualité B
3,Ski occasion Rossignol Experience 86 Basalt 20...,"309,00 EUR",https://freeglisse.com/fr/ski-occasion-adulte-...,Qualité B
4,Ski occasion Volkl Racetiger GS + fixations,"309,00 EUR",https://freeglisse.com/fr/ski-occasion-adulte-...,Qualité B
...,...,...,...,...
306,Ski FILLE occasion junior toutes marques à 29 ...,"29,00 EUR",https://freeglisse.com/fr/ski-occasion-junior-...,Qualité B
307,Ski occasion junior toutes marques à 29 € + Fi...,"29,00 EUR",https://freeglisse.com/fr/ski-occasion-junior-...,Qualité B
308,Ski occasion junior Rossignol Comp J / 9J tout...,"29,00 EUR",https://freeglisse.com/fr/ski-occasion-junior-...,Qualité B
309,Mini Ski occasion Snowblade Toutes Marques Tou...,"19,00 EUR",https://freeglisse.com/fr/mini-ski-occasion-ad...,Qualité B


#### Table qualité C

In [28]:
url_c = "https://freeglisse.com/fr/12-ski-occasion/s-1/etat_du_materiel-qualite_c"
reponse_c = requests.get(url_c)
html_content_c = reponse_c.text
soup_c = BeautifulSoup(html_content_c, "html.parser")

def table_qualite_c():
	base_url = "https://freeglisse.com/fr/12-ski-occasion/s-1/etat_du_materiel-qualite_c?page={}&order=product.price.desc"
	current_page = 1

	df_global_c = pd.DataFrame(columns=['nom', 'prix', 'lien'])

	while True:
		url = base_url.format(current_page)

		response = requests.get(url)
		if response.status_code != 200:
			print("Erreur lors de la requête HTTP")
			break

		soup = BeautifulSoup(response.text, 'html.parser')

		balises_h2 = soup.find_all("h2", {"class": "h3 product-title"})
		noms_produits = [balise_h2.a.text.strip() for balise_h2 in balises_h2]

		prices = [unidecode(span.get_text().strip()) for span in soup.find_all("span", {"class": "price"})]

		product_links = [h2_tag.a.get("href") for h2_tag in balises_h2]

		data = {'nom': noms_produits, 'prix': prices, 'lien': product_links}
		df_current = pd.DataFrame(data)

		df_global_c = pd.concat([df_global_c, df_current], ignore_index=True)

		next_page_link = soup.find("a", {"rel": "next", "class": "next js-search-link"})
		if not next_page_link:
			break

		current_page += 1
	return df_global_c

df_c = table_qualite_c()

df_c['Qualité'] = 'Qualité C'
df_c

,nom,prix,lien,Qualité
0,Ski occasion Atomic Redster S9 + fixations,"349,00 EUR",https://freeglisse.com/fr/ski-occasion-adulte-...,Qualité C
1,Ski occasion Elan Porsche Design Ace + fixations,"259,00 EUR",https://freeglisse.com/fr/ski-occasion-adulte-...,Qualité C
2,Ski occasion Black Crows ORB + fixations,"219,00 EUR",https://freeglisse.com/fr/ski-occasion-adulte-...,Qualité C
3,Ski occasion Head Premium Edition + fixations,"209,00 EUR",https://freeglisse.com/fr/ski-occasion-adulte-...,Qualité C
4,Ski occasion Elan Amphibio Porsche Design + fi...,"189,00 EUR",https://freeglisse.com/fr/ski-occasion-adulte-...,Qualité C
...,...,...,...,...
127,Ski de fond occasion Toutes marques + fixation...,"19,00 EUR",https://freeglisse.com/fr/ski-de-fond-occasion...,Qualité C
128,Ski occasion Exonde XO H21 + fixations,"19,00 EUR",https://freeglisse.com/fr/ski-occasion-adulte-...,Qualité C
129,Ski de fond occasion Toutes marques + fixation...,"15,00 EUR",https://freeglisse.com/fr/ski-de-fond-occasion...,Qualité C
130,Ski de fond occasion Toutes marques + fixatio...,"9,00 EUR",https://freeglisse.com/fr/ski-de-fond-occasion...,Qualité C


#### MERGE A,B et C pour All_DF

In [29]:
frames = [df_a, df_b, df_c]
all_df = pd.concat(frames)
display(all_df)

,nom,prix,lien,Qualité
0,Ski occasion Blizzard Anomaly 102 + fixations,"649,00 EUR",https://freeglisse.com/fr/ski-test-all-mountai...,Qualité A
1,Ski occasion Blizzard Anomaly 94 + fixations,"629,00 EUR",https://freeglisse.com/fr/ski-test-all-mountai...,Qualité A
2,Ski occasion Blizzard Anomaly 84 + fixations,"549,00 EUR",https://freeglisse.com/fr/ski-test-all-mountai...,Qualité A
3,Ski occasion Blizzard Thunderbird R15 + fixations,"519,00 EUR",https://freeglisse.com/fr/ski-test-piste/20267...,Qualité A
4,Ski occasion Blizzard Thunderbird R15 WB + fix...,"499,00 EUR",https://freeglisse.com/fr/ski-test-piste/20269...,Qualité A
...,...,...,...,...
127,Ski de fond occasion Toutes marques + fixation...,"19,00 EUR",https://freeglisse.com/fr/ski-de-fond-occasion...,Qualité C
128,Ski occasion Exonde XO H21 + fixations,"19,00 EUR",https://freeglisse.com/fr/ski-occasion-adulte-...,Qualité C
129,Ski de fond occasion Toutes marques + fixation...,"15,00 EUR",https://freeglisse.com/fr/ski-de-fond-occasion...,Qualité C
130,Ski de fond occasion Toutes marques + fixatio...,"9,00 EUR",https://freeglisse.com/fr/ski-de-fond-occasion...,Qualité C


In [30]:
all_df.groupby(['lien', 'Qualité']).count().reset_index().sort_values(by='nom', ascending=False).drop_duplicates(subset=['lien']).value_counts('Qualité')

Qualité
Qualité B    303
Qualité A    290
Qualité C    129
Name: count, dtype: int64

In [12]:
all_df

,nom,prix,lien,Qualité
0,Ski occasion junior Rossignol Star Wars + fixa...,"49,00 EUR",https://freeglisse.com/fr/ski-occasion-junior-...,Qualité A
1,Ski occasion Rossignol Nova 6 + fixations,"169,00 EUR",https://freeglisse.com/fr/ski-occasion-femme-l...,Qualité A
2,Ski occasion junior Rossignol Experience pro +...,"69,00 EUR",https://freeglisse.com/fr/ski-occasion-junior-...,Qualité A
3,Ski occasion Rossignol Sender 104 Ti 2024 + F...,"339,00 EUR",https://freeglisse.com/fr/ski-occasion-adulte-...,Qualité A
4,Ski occasion Rossignol Sender 94 Ti + fixations,"319,00 EUR",https://freeglisse.com/fr/accueil/18915-431270...,Qualité A
5,Ski occasion junior Rossignol Frozen + fixations,"69,00 EUR",https://freeglisse.com/fr/ski-occasion-junior-...,Qualité A
6,Ski occasion Salomon XDR 78 STR + fixations,"129,00 EUR",https://freeglisse.com/fr/ski-occasion-adulte-...,Qualité A
7,Ski occasion junior Rossignol Hero J + fixations,"59,00 EUR",https://freeglisse.com/fr/ski-occasion-junior-...,Qualité A
8,Ski occasion junior Rossignol Star Wars Baby +...,"49,00 EUR",https://freeglisse.com/fr/ski-occasion-junior-...,Qualité A
9,Ski occasion junior Rossignol fun Girl + fixat...,"55,00 EUR",https://freeglisse.com/fr/ski-occasion-junior-...,Qualité A


### Boucle V1 (SANS les stocks) pour obtenir toutes les informations sur toutes les pages 

In [31]:
def TOUFIK():

    # CREATION DES LISTES
    nom_produit_list = []
    marque_produit_list = []
    prix_produit_list = []
    ref_produit_list = []
    type_pratique_list = []
    utilisateur_list = []
    niveau_list = []
    colori_list = []
    nb_avis_list = []
    note_produit_list = []
    availability_msg_list = []

    for link in all_df['lien']:
        url_p = link
        reponse_p = requests.get(url_p)
        html_content_p = reponse_p.text
        soup_p = BeautifulSoup(html_content_p, "html.parser")

        # PART 2 - récupérer les infos de la page produit
        # Le NOM du produit
        nom_produit = soup_p.find_all("h1", {"class": "h1 product-detail-name"})
        nom_produit_list.append(nom_produit[0].get_text().strip() if nom_produit else np.nan)

        # La MARQUE du produit
        marque_produit = soup_p.find("img", {"class": "img img-thumbnail manufacturer-logo"})
        marque_produit_list.append(marque_produit.get("alt", "") if marque_produit else np.nan)

        # Le PRIX du produit
        prix_produit = soup_p.find_all("span", {"class": "current-price-value"})
        prix_produit_list.append(unidecode(prix_produit[0].get_text().strip()) if prix_produit else np.nan)

        # La REF du produit
        ref_produit = soup_p.find("div", {"class": "product-reference rb-tag-cate"})
        ref_produit_list.append(
            (ref_produit.text.strip().replace("announcement Référence: ", "").strip()) if ref_produit else np.nan)

        # La DESCRIPTION du produit
        descrip_prod_1 = soup_p.find_all("dd", {"class": "value"})
        type_pratique_list.append(descrip_prod_1[0].get_text().strip() if descrip_prod_1 else 'Null')
        utilisateur_list.append(descrip_prod_1[1].get_text().strip() if descrip_prod_1 and len(descrip_prod_1) > 1 else np.nan)
        niveau_list.append(descrip_prod_1[2].get_text().strip() if descrip_prod_1 and len(descrip_prod_1) > 2 else np.nan)
        colori_list .append(descrip_prod_1[3].get_text().strip() if descrip_prod_1 and len(descrip_prod_1) > 3 else np.nan)

        #Le NB d'avis
        nb_avis_element = soup_p.find("p", {"class": "netreviews_subtitle"})
        nb_avis = nb_avis_element.find("span", {"id": "reviewCount"}).text.strip() if nb_avis_element else np.nan
        nb_avis_list.append(nb_avis)

        # La NOTE du 5
        note_produit_element = soup_p.find("p", {"class": "netreviews_note_generale"})
        note_produit = note_produit_element.text.strip().split('/')[0] if note_produit_element else np.nan
        note_produit_list.append(note_produit)

        # La DISPONIBILITÉ du produit
        availability_msg = soup_p.find('span', {'id': 'availability_message'})
        availability_msg_list.append(availability_msg.get_text().strip() if availability_msg else np.nan)

    # APPEND AUX LISTES
    all_df['nom_produit'] = nom_produit_list
    all_df['marque_produit'] = marque_produit_list
    all_df['prix_produit'] = prix_produit_list
    all_df['ref_produit'] = ref_produit_list
    all_df['type_pratique'] = type_pratique_list
    all_df['utilisateur'] = utilisateur_list
    all_df['niveau'] = niveau_list
    all_df['colori'] = colori_list
    all_df['nb_avis'] = nb_avis_list
    all_df['note_produit'] = note_produit_list
    all_df['availability_msg'] = availability_msg_list
    
TOUFIK()
all_df

,nom,prix,lien,Qualité,nom_produit,marque_produit,prix_produit,ref_produit,type_pratique,utilisateur,niveau,colori,nb_avis,note_produit,availability_msg
0,Ski occasion Blizzard Anomaly 102 + fixations,"649,00 EUR",https://freeglisse.com/fr/ski-test-all-mountai...,Qualité A,Ski occasion Blizzard Anomaly 102 + fixations,Blizzard,"649,00 EUR",20266_e06,Freeride,Mixte,Performant,Violet,NaN,NaN,Disponible
1,Ski occasion Blizzard Anomaly 94 + fixations,"629,00 EUR",https://freeglisse.com/fr/ski-test-all-mountai...,Qualité A,Ski occasion Blizzard Anomaly 94 + fixations,Blizzard,"629,00 EUR",20264_e02,All mountain,Homme,Performant,Gris,NaN,NaN,Disponible
2,Ski occasion Blizzard Anomaly 84 + fixations,"549,00 EUR",https://freeglisse.com/fr/ski-test-all-mountai...,Qualité A,Ski occasion Blizzard Anomaly 84 + fixations,Blizzard,"549,00 EUR",20265_e04,All mountain,Mixte,Performant,Gris,NaN,NaN,Disponible
3,Ski occasion Blizzard Thunderbird R15 + fixations,"519,00 EUR",https://freeglisse.com/fr/ski-test-piste/20267...,Qualité A,Ski occasion Blizzard Thunderbird R15 + fixations,Blizzard,"519,00 EUR",20267_e08,Piste,Mixte,Performant,Noir,NaN,NaN,Disponible
4,Ski occasion Blizzard Thunderbird R15 WB + fix...,"499,00 EUR",https://freeglisse.com/fr/ski-test-piste/20269...,Qualité A,Ski occasion Blizzard Thunderbird R15 WB + fix...,Blizzard,"499,00 EUR",20269_e12,Piste,Mixte,Performant,Noir,NaN,NaN,Disponible
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,Ski de fond occasion Toutes marques + fixation...,"19,00 EUR",https://freeglisse.com/fr/ski-de-fond-occasion...,Qualité C,Ski de fond occasion Toutes marques + fixation...,Toutes marques,"19,00 EUR",18999_mz_l08,Alternatif,Mixte,Loisir,Gris,1,5,Disponible
128,Ski occasion Exonde XO H21 + fixations,"19,00 EUR",https://freeglisse.com/fr/ski-occasion-adulte-...,Qualité C,Ski occasion Exonde XO H21 + fixations,Exonde,"229,00 EUR",19899_f01,All mountain,Mixte,Performant,Bleu,NaN,NaN,Disponible
129,Ski de fond occasion Toutes marques + fixation...,"15,00 EUR",https://freeglisse.com/fr/ski-de-fond-occasion...,Qualité C,Ski de fond occasion Toutes marques + fixation...,Toutes marques,"15,00 EUR",15468_mz_l20,Alternatif,Mixte,Loisir,Blanc,11,4.8,Disponible
130,Ski de fond occasion Toutes marques + fixatio...,"9,00 EUR",https://freeglisse.com/fr/ski-de-fond-occasion...,Qualité C,Ski de fond occasion Toutes marques + fixatio...,Toutes marques,"9,00 EUR",15483_mz_k17,Alternatif,Mixte,Débutant,Blanc,6,4.7,Disponible


In [36]:
all_df.drop_duplicates(inplace=True)

In [37]:
#CheakPoint
all_df.to_csv('/Users/yacinehamdi/Downloads/freegliss.csv')
#df = pd.read_csv("/Users/yacinehamdi/Downloads/freegliss.csv")

### Boucle V2 (AVEC les stocks) pour obtenir toutes les informations sur toutes les pages 

Cette boucle peut-être très longue (environ 50 heures pour 2000 articles)
Nous allons donc tester sur les 10 premiers articles

In [45]:
#test sur 10 articles
test = df_a.sample(10)
test

,nom,prix,lien,Qualité
282,Ski occasion junior Rossignol Star Wars + fixa...,"49,00 EUR",https://freeglisse.com/fr/ski-occasion-junior-...,Qualité A
58,Ski occasion Rossignol Experience 80x + fixations,"229,00 EUR",https://freeglisse.com/fr/ski-occasion-femme-a...,Qualité A
216,Ski occasion Roxy + Fixations,"65,00 EUR",https://freeglisse.com/fr/ski-occasion-femme-l...,Qualité A
6,Ski occasion Blizzard Thunderbird R18 + fixations,"479,00 EUR",https://freeglisse.com/fr/ski-test-piste/20268...,Qualité A
70,Ski occasion K2 Pinnacle 95 + fixations,"199,00 EUR",https://freeglisse.com/fr/ski-occasion-freerid...,Qualité A
188,Ski occasion junior Volkl Racetiger + fixations,"79,00 EUR",https://freeglisse.com/fr/ski-occasion-junior-...,Qualité A
235,Ski occasion Tecnopro XR 200 + fixations,"59,00 EUR",https://freeglisse.com/fr/ski-occasion-adulte-...,Qualité A
60,Ski occasion Volkl 90 Eight W + fixations,"229,00 EUR",https://freeglisse.com/fr/ski-occasion-freerid...,Qualité A
93,Ski occasion Atomic Redster XTI + fixations,"169,00 EUR",https://freeglisse.com/fr/ski-occasion-adulte-...,Qualité A
12,Ski occasion Volkl Racetiger GS + fixations,"349,00 EUR",https://freeglisse.com/fr/ski-occasion-adulte-...,Qualité A


In [46]:
def TOUFIK():
    # CREATION DES LISTES
    nom_produit_list = []
    marque_produit_list = []
    prix_produit_list = []
    ref_produit_list = []
    type_pratique_list = []
    utilisateur_list = []
    niveau_list = []
    colori_list = []
    nb_avis_list = []
    note_produit_list = []
    availability_msg_list = []
    stock_list = []  # Liste pour le stock

    # Initialiser le WebDriver (spécifiez le chemin vers le chromedriver si nécessaire)
    driver = webdriver.Chrome()

    for link in test['lien']:
        url_p = link
        reponse_p = requests.get(url_p)
        html_content_p = reponse_p.text
        soup_p = BeautifulSoup(html_content_p, "html.parser")

        # PART 2 - récupérer les infos de la page produit
        # Le NOM du produit
        nom_produit = soup_p.find_all("h1", {"class": "h1 product-detail-name"})
        nom_produit_list.append(nom_produit[0].get_text().strip() if nom_produit else np.nan)

        # La MARQUE du produit
        marque_produit = soup_p.find("img", {"class": "img img-thumbnail manufacturer-logo"})
        marque_produit_list.append(marque_produit.get("alt", "") if marque_produit else np.nan)

        # Le PRIX du produit
        prix_produit = soup_p.find_all("span", {"class": "current-price-value"})
        prix_produit_list.append(unidecode(prix_produit[0].get_text().strip()) if prix_produit else np.nan)

        # La REF du produit
        ref_produit = soup_p.find("div", {"class": "product-reference rb-tag-cate"})
        ref_produit_list.append(
            (ref_produit.text.strip().replace("announcement Référence: ", "").strip()) if ref_produit else np.nan)

        # La DESCRIPTION du produit
        descrip_prod_1 = soup_p.find_all("dd", {"class": "value"})
        type_pratique_list.append(descrip_prod_1[0].get_text().strip() if descrip_prod_1 else 'Null')
        utilisateur_list.append(descrip_prod_1[1].get_text().strip() if descrip_prod_1 and len(descrip_prod_1) > 1 else np.nan)
        niveau_list.append(descrip_prod_1[2].get_text().strip() if descrip_prod_1 and len(descrip_prod_1) > 2 else np.nan)
        colori_list.append(descrip_prod_1[3].get_text().strip() if descrip_prod_1 and len(descrip_prod_1) > 3 else np.nan)

        # Le NB d'avis
        nb_avis_element = soup_p.find("p", {"class": "netreviews_subtitle"})
        nb_avis = nb_avis_element.find("span", {"id": "reviewCount"}).text.strip() if nb_avis_element else np.nan
        nb_avis_list.append(nb_avis)

        # La NOTE du produit
        note_produit_element = soup_p.find("p", {"class": "netreviews_note_generale"})
        note_produit = note_produit_element.text.strip().split('/')[0] if note_produit_element else np.nan
        note_produit_list.append(note_produit)

        # Ouvrir la page avec Selenium pour récupérer le stock
        driver.get(url_p)

        # Attendre que la page soit complètement chargée
        wait = WebDriverWait(driver, 10)

        # Variable pour stocker le maximum de produits en stock
        max_stock = 1  # Commencer avec une quantité de 1
        stock_found = False  # Variable pour détecter si le stock est trouvé

        try:
            # Localiser le champ de quantité
            quantity_input = wait.until(EC.visibility_of_element_located((By.ID, "quantity_wanted")))

            while True:
                # Utiliser JavaScript pour définir la valeur du champ de quantité
                driver.execute_script(f"arguments[0].value = {max_stock};", quantity_input)
                
                # Simuler un événement 'change' sur le champ de quantité
                driver.execute_script("var event = new Event('change', { bubbles: true }); arguments[0].dispatchEvent(event);", quantity_input)
                        # Faire défiler vers le haut de la page
                driver.execute_script("window.scrollTo(1, 1);")
                time.sleep(1)  # Petite pause pour s'assurer que le scroll est terminé
                # Attendre un peu pour que la page traite l'événement
                time.sleep(2)  # Ajustez cette durée si nécessaire
                # Rechercher le message de disponibilité à chaque itération
                try:
                    # Localiser à nouveau le message de disponibilité
                    availability_message = wait.until(
                        EC.visibility_of_element_located((By.XPATH, "//span[@id='availability_message']"))
                    )
                    availability_text = availability_message.text
                    print(f"Message de disponibilité : {availability_text}")

                    # Vérifier si le stock est insuffisant
                    if "pas assez" in availability_text.lower():
                        stock_list.append(max_stock - 1)
                        stock_found = True
                        break

                except Exception as e:
                    print(f"Erreur lors de la vérification du message : {str(e)}")
                    break
                
                # Si le message est valide, augmenter la quantité
                max_stock += 1

                # Défiler pour que l'élément de quantité soit visible
                driver.execute_script("arguments[0].scrollIntoView(true);", quantity_input)
                time.sleep(0.5)  # Petite pause pour s'assurer que l'élément est visible

            # Si aucun stock n'est trouvé, ajouter np.nan
            if not stock_found:
                stock_list.append(np.nan)

        except Exception as e:
            print(f"Erreur lors de la vérification du stock : {e}")
            stock_list.append(np.nan)  # En cas d'erreur, on ajoute np.nan

    # APPEND AUX LISTES
    test['nom_produit'] = nom_produit_list
    test['marque_produit'] = marque_produit_list
    test['prix_produit'] = prix_produit_list
    test['ref_produit'] = ref_produit_list
    test['type_pratique'] = type_pratique_list
    test['utilisateur'] = utilisateur_list
    test['niveau'] = niveau_list
    test['colori'] = colori_list
    test['nb_avis'] = nb_avis_list
    test['note_produit'] = note_produit_list
    #test['availability_msg'] = availability_msg_list
    test['stock'] = stock_list  # Ajout des stocks à la DataFrame

    # Fermer le WebDriver à la fin
    driver.quit()

TOUFIK()
display(test)

Message de disponibilité : Disponible
Message de disponibilité : Il n'y a pas assez de produits en stock.
Message de disponibilité : Disponible
Message de disponibilité : Il n'y a pas assez de produits en stock.
Message de disponibilité : Disponible
Message de disponibilité : Il n'y a pas assez de produits en stock.
Message de disponibilité : Disponible
Message de disponibilité : Il n'y a pas assez de produits en stock.
Message de disponibilité : Disponible
Message de disponibilité : Disponible
Message de disponibilité : Il n'y a pas assez de produits en stock.
Message de disponibilité : Disponible
Message de disponibilité : Disponible
Message de disponibilité : Disponible
Message de disponibilité : Disponible
Message de disponibilité : Il n'y a pas assez de produits en stock.
Message de disponibilité : Disponible
Message de disponibilité : Il n'y a pas assez de produits en stock.
Message de disponibilité : Disponible
Message de disponibilité : Il n'y a pas assez de produits en stock.


,nom,prix,lien,Qualité,nom_produit,marque_produit,prix_produit,ref_produit,type_pratique,utilisateur,niveau,colori,nb_avis,note_produit,stock
282,Ski occasion junior Rossignol Star Wars + fixa...,"49,00 EUR",https://freeglisse.com/fr/ski-occasion-junior-...,Qualité A,Ski occasion junior Rossignol Star Wars + fixa...,Rossignol,"49,00 EUR",11445_a40_vador,Piste,Junior,Débutant,Blanc,19,4.6,1
58,Ski occasion Rossignol Experience 80x + fixations,"229,00 EUR",https://freeglisse.com/fr/ski-occasion-femme-a...,Qualité A,Ski occasion Rossignol Experience 80x + fixations,Rossignol,"229,00 EUR",20271_o16,All mountain,Femme,Loisir sport,Bleu,NaN,NaN,1
216,Ski occasion Roxy + Fixations,"65,00 EUR",https://freeglisse.com/fr/ski-occasion-femme-l...,Qualité A,Ski occasion Roxy + Fixations,Roxy,"55,00 EUR",12246_i12,Piste,Femme,Loisir,Rose,1,5,1
6,Ski occasion Blizzard Thunderbird R18 + fixations,"479,00 EUR",https://freeglisse.com/fr/ski-test-piste/20268...,Qualité A,Ski occasion Blizzard Thunderbird R18 + fixations,Blizzard,"479,00 EUR",20268_e10_orange,Piste,Mixte,Performant,Orange,NaN,NaN,1
70,Ski occasion K2 Pinnacle 95 + fixations,"199,00 EUR",https://freeglisse.com/fr/ski-occasion-freerid...,Qualité A,Ski occasion K2 Pinnacle 95 + fixations,K2,"199,00 EUR",15234_f31_noirvert,Freeride,Mixte,Loisir sport,Vert,NaN,NaN,2
188,Ski occasion junior Volkl Racetiger + fixations,"79,00 EUR",https://freeglisse.com/fr/ski-occasion-junior-...,Qualité A,Ski occasion junior Volkl Racetiger + fixations,Volkl,"79,00 EUR",19687_a02,Racing,Junior,Compétition,Jaune,NaN,NaN,4
235,Ski occasion Tecnopro XR 200 + fixations,"59,00 EUR",https://freeglisse.com/fr/ski-occasion-adulte-...,Qualité A,Ski occasion Tecnopro XR 200 + fixations,Tecnopro,"59,00 EUR",13102_o35,Piste,Mixte,Loisir,Rouge,NaN,NaN,1
60,Ski occasion Volkl 90 Eight W + fixations,"229,00 EUR",https://freeglisse.com/fr/ski-occasion-freerid...,Qualité A,Ski occasion Volkl 90 Eight W + fixations,Volkl,"229,00 EUR",20031_n17,Freeride,Femme,Performant,Bleu,NaN,NaN,1
93,Ski occasion Atomic Redster XTI + fixations,"169,00 EUR",https://freeglisse.com/fr/ski-occasion-adulte-...,Qualité A,Ski occasion Atomic Redster XTI + fixations,Atomic,"169,00 EUR",10719_e10,Racing,Mixte,Performant,Bleu,NaN,NaN,1
12,Ski occasion Volkl Racetiger GS + fixations,"349,00 EUR",https://freeglisse.com/fr/ski-occasion-adulte-...,Qualité A,Ski occasion Volkl Racetiger GS + fixations,Volkl,"349,00 EUR",19842_j10,Racing,Homme,Compétition,Rouge,NaN,NaN,1
